# simGL

This python package simulates Genotype Likelihoods (GL) out of a haplotypic allele count matrix typically obtained from simulations.

NOTE: `simGL` is still in development and might have some bugs. Please, report them in issues if you were to encounter some or have suggestions for improvement.

### Installation


In [ ]:
git clone https://github.com/RacimoLab/simGL
cd simGL
pip install .

### Example


In [ ]:
import msprime
import numpy as np
import simGL

Then, using the former, we are going to obtain a `tree sequence` of the simulation from which we will extract the `haplotypic genotype matrix` and other relevant information below.

In [ ]:
ts = msprime.sim_ancestry(
    population_size=10_000,
    samples=10,
    sequence_length=100_000,
    recombination_rate=1.25e-8,
    random_seed=1234,
    record_provenance=False,
)
ts = msprime.sim_mutations(ts, rate=1.25e-8, random_seed=5678)

ts

We can extract the `genotype matrix` from the tree sequence. Note that by default, we are simulating diploid individuals and thus there are double number of haplotypes (20) than the number of individuals that we simulated (10). 

In [ ]:
gm = ts.genotype_matrix()
print(gm.shape)
gm

In this case, we have 141 SNPs.

Then, we can extract the reference and alternative alleles.

In [ ]:
ref = np.array([v.site.ancestral_state for v in ts.variants()])
alt = np.array([v.site.mutations[0].derived_state for v in ts.variants()])

print(ref.shape)
print(ref[:10])
print(alt.shape)
print(alt[:10])

Then, we can simulate allele read counts (`arc`). We must decide the mean read depth per haplotype and standard deviation. This will determine the shape of a normal distribution from which mean coverage per haplotype will be sampled. Alternatively, an array with the coverage means per haplotypic chromosomes can also be inputted. Then, the number of reads per haplotype per site will be sampled from a poison distribution. Finally, the reads of each allele (A, C, G and T) will be sampled from a multinomial distribution in which the probability of sampling an error allele is `e/3` and sampling the correct allele is `1-e`. Finally, `ploidy` columns will be sum together to form the allele read counts for an individual.

In [ ]:
e      = 0.05
ploidy = 2
arc = simGL.sim_allelereadcounts(gm = gm, ref = ref, alt = alt, 
                           mean_depth = 15., std_depth = 3., e = e, 
                           ploidy = ploidy, seed = 1234)
print(arc.shape)
arc

The output above matches the number of sites (first dimension) of the `genotype matrix`, has half the size of the second dimension (number of haplotypes are now number of individuals depending on ploidy) and has an additional dimension of size 4. Each value in this array correspond to the number of reads map to that particular site for a particular individual and the index of the value in the third dimension corresponds to each allele (in order: "A", "C", "G" and "T").

For example, the first individual is heterozygous T/C for the first site.

In [ ]:
print(ref[0], alt[0], gm[0, 0:2])

Correspondingly, we find the majority of alleles counts in the second and fourth positions of the array (corresponding to C and T respectively) and we also see that an error has been simulated for the first individual for the first site.

In [ ]:
arc[0, 0]

Furthermore, the coverage of this individual is close to the mean depth defined (15x per haplotypic chromosome).

In [ ]:
arc[0, 0].sum()

Finally, from the allele read counts, we can apply a GL model.

In [ ]:
GL  = simGL.allelereadcounts_to_GL(arc, e = e, ploidy = ploidy)
print(GL.shape)
GL

The output is given as the log(GL) normalized by substracting the value of the maximum likelihood genotype per site per individual. As you might noticed, the third dimension has a size of 10 since it is the number of possible genotypes for a diploid individual simulated here (in order: "AA", "AC", "AG", "AT", "CC", "CG", ..., "TT").

Thus, the first individual must have a value of 0 in the seventh value in this array.

In [ ]:
GL[0, 0]